<a href="https://colab.research.google.com/github/AditPradana36/GSV_downloader_bylinks/blob/main/GSV_downloader_bylinks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🗺️ Google Street View Downloader by A List of Links

This notebook lets you download **Google Street View images** from a list of URLs stored in an Excel file.  
It supports:
- 📌 **Heading slices** (download from multiple heading ranges)
- 🌐 **Panorama stitch** (combine into a single wide image)
- 📝 **Metadata export** (including capture date: year & month)

## ⚙️ How to Use This Notebook

### 1. 🔑 Get Your API Key
1. Go to the [Google Cloud Console](https://console.cloud.google.com/).
2. Create a project and enable:
   - **Street View Static API**
   - **Street View Image API**
3. Generate an **API Key** and paste it in the `api_key` field below.

⚠️ **Important:** Ensure billing is enabled, otherwise the API may block your requests.

### 2. 📂 Prepare Your Excel File
- Make an Excel file (e.g., `trylink.xlsx`) with at least **two columns**:
  - `id` → unique identifier (e.g., `1, 2, 3`)
  - `link` → Street View URL (copied from Google Maps)


Upload the file into Colab (default path: `/content/YOUR_FILE.xlsx`).




In [ ]:
# ============================================
# 🛠 Install dependencies
# ============================================
!pip install requests pillow openpyxl

import os
import re
import csv
import datetime
import requests
from io import BytesIO
from PIL import Image
from google.colab import files
import pandas as pd

### 3. ⚙️ Set Parameters
- `api_key`: Your Google API Key.
- `xlsx_path`: Path to your Excel file with links.
- `output_folder`: Where to save the downloaded images.
- `download_mode`: Choose between:
  - `"headings"` → download slices per heading range.
  - `"panorama"` → download and stitch into one panorama.
  - `"both"` → do both.
- `heading_ranges`: Define ranges for headings (default = 0–360 split into 4 parts).
- `image_size`: Output image size (e.g., `640x640`).
- `fov`: Field of View (recommended: `90`).
- `pitch`: Camera tilt angle (default: `0`).
- `metadata_output`: Where to save the CSV file with metadata.

In [ ]:
# ============================================
# 📌 Parameters - Edit these using Colab form
# ============================================
api_key = "YOUR_API_KEY"   # @param {type:"string"}
xlsx_path = "/content/YOUR_FILE.xlsx"  # @param {type:"string"}
output_folder = "/content"  # @param {type:"string"}

download_mode = "panorama"   # @param ["headings", "panorama", "both"]

# If using heading slices, define ranges here
heading_ranges = [(0,90), (90,180), (180,270), (270,360)]  # @param

image_size = "640x640"    # @param {type:"string"}
fov = 90                  # @param {type:"integer"}
pitch = 0                 # @param {type:"integer"}
metadata_output = "/content/metadata_from_links.csv"  # @param {type:"string"}

### 4. ▶️ Run the Notebook
- Run **each cell in order**.
- Images will be saved inside your chosen `output_folder`.
- A metadata CSV will be generated with:
  - `image_id`
  - `filename`
  - `pano_id`
  - `heading_range`
  - `capture_year`
  - `capture_month`
  - `status`
  - `download_timestamp`

In [ ]:
# ============================================
# 📦 Helper Functions
# ============================================
def extract_panoid_from_url(url):
    """Extract pano_id from Google Maps Street View URL"""
    match = re.search(r"!1s([a-zA-Z0-9_\-]+)!2e0", url)
    if match:
        return match.group(1)
    return None

def get_metadata_from_panoid(panoid):
    """Query Street View Metadata API for capture date (year, month), lat, lon"""
    base_url = "https://maps.googleapis.com/maps/api/streetview/metadata"
    params = {
        "pano": panoid,
        "key": api_key
    }
    r = requests.get(base_url, params=params)
    if r.status_code == 200:
        data = r.json()
        year, month = None, None
        lat, lon = None, None

        if "date" in data:
            try:
                year, month = data["date"].split("-")
                year, month = int(year), int(month)
            except:
                year, month = None, None

        if "location" in data:
            lat = data["location"].get("lat")
            lon = data["location"].get("lng")

        return year, month, lat, lon

    return None, None, None, None


def download_streetview_from_panoid(panoid, output_folder=".", image_id="image",
                                    mode="headings", heading_ranges=None):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    base_url = "https://maps.googleapis.com/maps/api/streetview"
    filenames = []

    # ---- Case 1: Individual headings ----
    if mode in ["headings", "both"]:
        for start, end in heading_ranges:
            heading = (start + end) // 2   # midpoint of range
            params = {
                'size': image_size,
                'fov': fov,
                'pitch': pitch,
                'key': api_key,
                'pano': panoid,
                'heading': heading
            }
            response = requests.get(base_url, params=params)
            if response.status_code != 200:
                print(f"❌ Failed at heading {heading} for panoid={panoid} | {response.text}")
                continue
            filename = f"id_{image_id}_{start}-{end}.jpg"
            filepath = os.path.join(output_folder, filename)
            with open(filepath, 'wb') as f:
                f.write(response.content)
            filenames.append((filename, f"{start}-{end}"))

    # ---- Case 2: Panorama (stitched) ----
    if mode in ["panorama", "both"]:
        imgs = []
        for start, end in heading_ranges:
            heading = (start + end) // 2
            params = {
                'size': image_size,
                'fov': fov,
                'pitch': pitch,
                'key': api_key,
                'pano': panoid,
                'heading': heading
            }
            response = requests.get(base_url, params=params)
            if response.status_code != 200:
                print(f"❌ Failed at heading {heading} for panoid={panoid} | {response.text}")
                continue
            imgs.append(Image.open(BytesIO(response.content)))

        if imgs:
            total_width = sum(img.width for img in imgs)
            max_height = max(img.height for img in imgs)
            panorama_img = Image.new("RGB", (total_width, max_height))
            x_offset = 0
            for img in imgs:
                panorama_img.paste(img, (x_offset, 0))
                x_offset += img.width

            filename = f"id_{image_id}_panorama.jpg"
            filepath = os.path.join(output_folder, filename)
            panorama_img.save(filepath)
            filenames.append((filename, "stitched"))

    return filenames if filenames else None

### 5. 📤 Download Results
- Images: check inside the `output_folder`.
- Metadata CSV: auto-downloaded at the end.

In [ ]:
# ============================================
# 🔁 Process All Rows in Excel
# ============================================
os.makedirs(output_folder, exist_ok=True)

with open(metadata_output, 'w', newline='') as meta_file:
    fieldnames = ['image_id', 'filename', 'pano_id', 'heading_range',
                  'capture_year', 'capture_month', 'latitude', 'longitude',
                  'status', 'download_timestamp']
    writer = csv.DictWriter(meta_file, fieldnames=fieldnames)
    writer.writeheader()

    df = pd.read_excel(xlsx_path)

    for idx, row in df.iterrows():
        image_id = str(row['id'])
        link = row['link']

        panoid = extract_panoid_from_url(link)
        if not panoid:
            print(f"❌ Could not extract panoid for ID {image_id}")
            continue

        # Get capture date (year, month) and lat/lon
        capture_year, capture_month, lat, lon = get_metadata_from_panoid(panoid)

        results = download_streetview_from_panoid(
            panoid,
            output_folder=output_folder,
            image_id=image_id,
            mode=download_mode,
            heading_ranges=heading_ranges
        )
        if not results:
            continue

        for fn, hr in results:
            writer.writerow({
                'image_id': image_id,
                'filename': fn,
                'pano_id': panoid,
                'heading_range': hr,
                'capture_year': capture_year,
                'capture_month': capture_month,
                'latitude': lat,
                'longitude': lon,
                'status': "OK",
                'download_timestamp': datetime.datetime.now().isoformat()
            })

print(f"✅ All images downloaded to `{output_folder}`")
print(f"📄 Metadata saved to `{metadata_output}`")

✅ All images downloaded to `/content`
📄 Metadata saved to `/content/metadata_from_links.csv`
